In [14]:
%pip install faker pandas pyarrow

import pandas as pd
import numpy as np
from faker import Faker
import random
import os

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [15]:
# Función Faker
faker = Faker()
np.random.seed(0)

# (BRONZE)
# Creación carpeta Bronze
os.makedirs("bronze", exist_ok=True)

In [16]:
# (BRONZE)
# Dimensión Fecha
def gen_dim_fecha(inicio="2024-01-01", fin="2024-12-31"):
  fechas = pd.date_range(start=inicio, end=fin, freq="D")
  df = pd.DataFrame({
      "fecha_id": range(1, len(fechas)+1),
      "dia": fechas.day,
      "mes": fechas.month,
      "año": fechas.year,
      "dia_semana": fechas.day_name(),
      "si_es_fin_de_semana": fechas.weekday >= 5
  })

  df.to_parquet("bronze/fechas.parquet", index=False)
  return df

In [17]:
# (BRONZE)
# Dimensión Producto
def gen_dim_producto(n=200):
    productos_pc = [
        "Teclado", "Mouse", "Monitor", "CPU", "Placa Madre", "Memoria RAM",
        "Disco Duro", "SSD", "Fuente de Poder", "Tarjeta Gráfica", "Gabinete",
        "Ventilador", "Webcam", "Auriculares", "Micrófono", "Joystick", "Router",
        "Adaptador WiFi", "Docking Station", "Impresora", "Scanner"
    ]
    productos = []
    for i in range(1, n + 1):
        productos.append({
            "producto_id": i,
            "nombre_producto": random.choice(productos_pc),
            "marca": faker.word().capitalize() + " " + faker.word().capitalize(),
            "precio_base": round(random.uniform(10, 800), 2)
        })

    df = pd.DataFrame(productos)
    df.to_parquet("bronze/productos.parquet", index=False)
    return df

In [18]:
# (BRONZE)
# Dimensión Tienda
def gen_dim_tienda(n=3000):
    tiendas=[]

    # Diccionario con estados y ciudades
    estados_y_ciudades = {
      # México (solo algunos para ejemplo)
      "Ciudad de México": ["Ciudad de México"],
      "Jalisco": ["Guadalajara"],
      "Nuevo León": ["Monterrey"],
      "Quintana Roo": ["Cancún"],
      "Puebla": ["Puebla"],

      # Argentina
      "Buenos Aires": ["Buenos Aires"],

      # España
      "Madrid": ["Madrid"],
      "Cataluña": ["Barcelona"],

      # Francia
      "Île-de-France": ["París"],

      # Estados Unidos
      "Nueva York": ["Nueva York"],

      # Canadá
      "Ontario": ["Toronto"],

      # Alemania
      "Baviera": ["Múnich"],

      # Brasil
      "São Paulo": ["São Paulo"],

      # India
      "Maharashtra": ["Mumbai"],

      # Italia
      "Lombardía": ["Milán"],

      # China
      "Guangdong": ["Cantón"],

      # Australia
      "Nueva Gales del Sur": ["Sídney"]
    }

    estados = list(estados_y_ciudades.keys())

    for i in range(1, n + 1):
        estado = random.choice(estados)
        ciudad = random.choice(estados_y_ciudades[estado])

        tiendas.append({
            "tienda_id": i,
            "nombre": faker.company(),
            "ciudad": ciudad,
            "estado": estado,
            "tipo": random.choice(["Nacional", "Internacional", "Transnacional"])
        })

    df = pd.DataFrame(tiendas)
    df.to_parquet("bronze/tiendas.parquet", index=False)
    return df

In [19]:
# (BRONZE)
# Tabla Factual Ventas
def gen_tabla_ventas(df_fecha=None, df_producto=None, df_tienda=None, num_registros=6000):

    # Aseguramos unicidad en productos y tiendas
    df_producto = df_producto.drop_duplicates(subset="producto_id")
    df_tienda = df_tienda.drop_duplicates(subset="tienda_id")

    # Creamos un diccionario para mapear producto_id a su precio base (para más rapido xd)
    precio_dict = df_producto.set_index("producto_id")["precio_base"].to_dict()

    ventas = []

    # Extraemos los id's para fechas, tiendas y productos para la selección aleatoria
    fechas_ids = df_fecha["fecha_id"].values
    tiendas_ids = df_tienda["tienda_id"].values
    productos_ids = df_producto["producto_id"].values

    # Generamos el numero de registros aleatorios de la tabla
    for i in range(num_registros):
        fecha_id = np.random.choice(fechas_ids)
        tienda_id = np.random.choice(tiendas_ids)
        producto_id = np.random.choice(productos_ids)

        precio_unitario = precio_dict[producto_id]

        cantidad = np.random.randint(1, 10)
        total = cantidad * precio_unitario

        ventas.append({
            "venta_id": i + 1,
            "fecha_id": fecha_id,
            "producto_id": producto_id,
            "tienda_id": tienda_id,
            "cantidad": cantidad,
            "precio_unitario": precio_unitario,
            "total": total
        })

    df = pd.DataFrame(ventas)
    df.to_parquet("bronze/ventas.parquet", index=False)
    return df

In [20]:
# (BRONZE)
# Creación de DataFrames
df_fecha = gen_dim_fecha()
df_producto = gen_dim_producto()
df_tienda = gen_dim_tienda()
df_ventas = gen_tabla_ventas(df_fecha, df_producto, df_tienda)
print("Datos Generados")

Datos Generados


In [21]:
# (SILVER)
# Creación carpeta Silver
os.makedirs("silver", exist_ok=True)

In [22]:
# (SILVER)
# Función de verificación de que NO hay fechas duplicadas
def silver_layer():
    #Leemos el parquet de Bronze
    df_ventas = pd.read_parquet("bronze/ventas.parquet")
    df_fechas = pd.read_parquet("bronze/fechas.parquet")
    df_tiendas = pd.read_parquet("bronze/tiendas.parquet")
    df_productos = pd.read_parquet("bronze/productos.parquet")

    # Eliminamos duplicados
    df_ventas = df_ventas.drop_duplicates()
    df_fechas_sin_duplicados = df_fechas.drop_duplicates(subset=['año', 'mes', 'dia'])

    # Limpiar columnas ciudad y estado en df_tiendas (si vienen como listas o arreglos), para evitar errores en Gold:
    for col in ['ciudad', 'estado']:
        if col in df_tiendas.columns:
            df_tiendas[col] = df_tiendas[col].apply(lambda x: x[0] if isinstance(x, (list, np.ndarray)) else x)

    # Verificamos también en productos si no hay listas o arreglos (por si acaso xd):
    for col in df_productos.columns:
        df_productos[col] = df_productos[col].apply(lambda x: x[0] if isinstance(x, (list, np.ndarray)) else x)

    # Incersión de los datos limpios en Silver
    df_silver = df_ventas.merge(df_fechas_sin_duplicados, on="fecha_id", how="left") \
                         .merge(df_tiendas, on="tienda_id", how="left") \
                         .merge(df_productos, on="producto_id", how="left")

    df_silver.to_parquet("silver/ventas_enriquecidas.parquet", index=False)
    return df_silver

In [23]:
# (SILVER)
# Creación de DataFrame
df_silver = silver_layer()
print("Datos Generados")

Datos Generados


In [24]:
# (GOLD)
# Creación carpeta Gold
os.makedirs("gold", exist_ok=True)

In [25]:
# (GOLD)
# Función de consultas
def gold_layer():
    #Leemos el parquet de Silver
    df_silver = pd.read_parquet("silver/ventas_enriquecidas.parquet")

    # Consulta 1: Total ventas por mes
    ventas_por_mes = df_silver.groupby("mes")["total"].sum().reset_index()
    ventas_por_mes.to_parquet("gold/ventas_por_mes.parquet", index=False)
    display(ventas_por_mes)

    # Consulta 2: Total ventas por ciudad
    ventas_por_ciudad = df_silver.groupby("ciudad")["total"].sum().reset_index()
    ventas_por_ciudad.to_parquet("gold/ventas_por_ciudad.parquet", index=False)
    display(ventas_por_ciudad)

    # Consulta 3: Producto más vendido (por cantidad)
    producto_top = df_silver.groupby("nombre_producto")["cantidad"].sum().reset_index() \
                    .sort_values(by="cantidad", ascending=False).head(1)
    producto_top.to_parquet("gold/producto_top.parquet", index=False)
    display(producto_top)

    # Consulta 4: Comparativa de ventas entre tiendas (por total vendido)
    ventas_por_tienda = df_silver.groupby("nombre")["total"].sum().reset_index()
    ventas_por_tienda.to_parquet("gold/ventas_por_tienda.parquet", index=False)
    display(ventas_por_tienda)

    return {
        "ventas_por_mes": ventas_por_mes,
        "ventas_por_ciudad": ventas_por_ciudad,
        "producto_top": producto_top,
        "ventas_por_tienda": ventas_por_tienda
    }

In [26]:
# (GOLD)
# Ejecución de Consultas
resultados = gold_layer()

,mes,total
0,1,1035365.05
1,2,801198.44
2,3,1100539.39
3,4,967740.28
4,5,982647.53
5,6,1031677.19
6,7,1115908.51
7,8,1018881.39
8,9,978265.88
9,10,935184.67


,ciudad,total
0,Barcelona,641710.69
1,Buenos Aires,722005.69
2,Cancún,714892.15
3,Cantón,674577.07
4,Ciudad de México,829312.10
5,Guadalajara,787659.25
6,Madrid,673507.66
7,Milán,650136.34
8,Monterrey,621871.66
9,Mumbai,739650.08


,nombre_producto,cantidad
18,Teclado,2525


,nombre,total
0,"Adams, Davis and Cross",3387.27
1,"Adams, Jordan and Lane",5306.75
2,"Adams, Morris and Armstrong",832.32
3,Adams-Blake,5315.76
4,Adams-Jennings,4706.83
...,...,...
2423,Zimmerman LLC,7691.05
2424,Zimmerman-Johnson,1757.44
2425,Zuniga Ltd,1235.86
2426,Zuniga and Sons,1115.31
